In [1]:
import real_estate_api

from typing import Union, List
from types import ModuleType
from beartype import beartype
import warnings

import numpy as np
import pandas as pd
import sys
import importlib
from math import radians, cos, sin, asin, sqrt

from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

import keys as k
keys = k.getKeys()

import pgeocode

'''
### For pgeocode to work, I am not going to pretend to know what this does ###
# https://stackoverflow.com/questions/27835619/urllib-and-ssl-certificate-verify-failed-error
import certifi
import os
import ssl

openssl_dir, openssl_cafile = os.path.split(
        ssl.get_default_verify_paths().openssl_cafile
)

os.chdir(openssl_dir)
relpath_to_certifi_cafile = os.path.relpath(certifi.where())

try:
    os.remove(openssl_cafile)
except FileNotFoundError:
    pass

os.symlink(relpath_to_certifi_cafile, openssl_cafile)
###
'''

'\n### For pgeocode to work, I am not going to pretend to know what this does ###\n# https://stackoverflow.com/questions/27835619/urllib-and-ssl-certificate-verify-failed-error\nimport certifi\nimport os\nimport ssl\n\nopenssl_dir, openssl_cafile = os.path.split(\n        ssl.get_default_verify_paths().openssl_cafile\n)\n\nos.chdir(openssl_dir)\nrelpath_to_certifi_cafile = os.path.relpath(certifi.where())\n\ntry:\n    os.remove(openssl_cafile)\nexcept FileNotFoundError:\n    pass\n\nos.symlink(relpath_to_certifi_cafile, openssl_cafile)\n###\n'

In [91]:
# This is going to be where I keep all the API endpoints.
from retrying import retry
from ediblepickle import checkpoint
from urllib.parse import quote
from typing import Literal
from beartype import beartype
from beartype.typing import Tuple
from datetime import datetime
import requests
import html
import string

RESULTS_PER_REQUEST_LIMIT = 42

USREALESTATE_API_HEADERS = {
    "X-RapidAPI-Key": keys['USRealEstate'],
    "X-RapidAPI-Host": "us-real-estate.p.rapidapi.com"
}

@beartype
@retry(stop_max_attempt_number=5)
@checkpoint(key=lambda args, kwargs: quote(args[0]) + '.pkl', work_dir='Saved Results/LocationSuggest/')
def get_LocationSuggest(
        search_keyword : str, 
        return_all     : bool = False
    ) -> dict:

    url = "https://us-real-estate.p.rapidapi.com/location/suggest"

    querystring = {
        "input":search_keyword
    }

    response = requests.request("GET", url, headers=USREALESTATE_API_HEADERS, params=querystring)
    response_json = response.json()

    return response_json if return_all else response_json['data'][0]

@beartype
@retry(stop_max_attempt_number=5)
@checkpoint(key=lambda args, kwargs: quote(args[0]) + '.pkl', work_dir='Saved Results/PropertyDetail/')
def get_PropertyDetail(
        property_id : str
    ) -> dict:
    
    if not isinstance(property_id, str):
        try:
            property_id = str(property_id)
        except:
            raise Exception('Could not convert input to string.')

    url = "https://us-real-estate.p.rapidapi.com/v2/property-detail"

    querystring = {
        "property_id":property_id
    }

    response = requests.request("GET", url, headers=USREALESTATE_API_HEADERS, params=querystring)
    return response.json()

@beartype
@retry(stop_max_attempt_number=5)
@checkpoint(key=lambda args, kwargs: quote(args[0]) + '.pkl', work_dir='Saved Results/PropertyValue/')
def get_PropertyValue(property_id : str) -> dict:
    url = "https://us-real-estate.p.rapidapi.com/for-sale/home-estimate-value"

    querystring = {
        "property_id":property_id
    }

    response = requests.request("GET", url, headers=USREALESTATE_API_HEADERS, params=querystring)
    return response.json()

def get_Properties(
        market_status : Literal['for_sale','sold'],
        city_state    : Tuple[str, str] = None,
        zip_code      : str = None,
        n_results     : int = None,
        property_type : str = 'single_family',
        offset        : int = 0,
        verbose       : bool = False
    ) -> dict:
    
    '''
    This function will return properties, either recently sold or currently on the market, 
    within a specific city and state or zip code.
    Required fields: market_status, EITHER city_state or zip_code.

    There is something strange about the way this is setup where checkpointing fails, 
    and am not sure why that is happening.
    For now, I will be checkpointing the parent function.

    Although offset might seem a bit strange to pass in here,I will be checking that there is enough samples requested.
    The first round of outputs will dictate if we need to query more sold/for sale homes.
    '''

    if city_state is None and zip_code is None:
        raise Exception('Either city_state or zip_code is required.')

    if city_state is not None and zip_code is not None:
        raise Exception('Both city_state and zip_code can not be populated.')

    '''
    Other query string parameters:
    sort = (default: relevant)|newest|lowest_price|highest_price|open_house_date|
                                price_reduced_date|largest_sqft|lot_size|sold_date
    price_min/max = $ USD
    beds_min/max = #
    bath_min/max = #
    property_type = multi_family|single_family|mobile|land|farm (I think we should just use : 'single_family')
    '''

    querystring = {
        "offset":str(offset),
        "limit":str(RESULTS_PER_REQUEST_LIMIT),
        "sort":"sold_date" if market_status == 'sold' else "newest",
        "property_type":property_type,
        "min_price":100
    }

    # This will be looking at city, state
    if zip_code is None:
        if market_status == 'sold': # Sold city, state
            url = "https://us-real-estate.p.rapidapi.com/sold-homes"
        else: # For sale city, state
            url = "https://us-real-estate.p.rapidapi.com/v2/for-sale"

        querystring.update({
            "state_code":city_state[1],
            "city":city_state[0]
        }) 
    else:
        if market_status == 'sold': # Sold zip code
            url = "https://us-real-estate.p.rapidapi.com/v2/sold-homes-by-zipcode"
            del querystring['min_price']
        else: # For sale zip code
            url = "https://us-real-estate.p.rapidapi.com/v2/for-sale-by-zipcode"

        querystring.update({
            "zipcode":zip_code,
        })

    geo_to_return, houses_to_return = query_url(
        n_results, 
        verbose, 
        querystring, 
        url, 
        zzzipcode = querystring.get('zipcode', '00000'),
        zzzcity = querystring.get('city', 'XXXXX'),
        zzzsort = querystring.get('sort')
    )

    return {
        'houses' : houses_to_return,
        'geo' : geo_to_return
    }

@beartype
@retry(stop_max_attempt_number=5)
@checkpoint(key=string.Template('${4}_${zzzcity}_${6}.csv'), work_dir='Saved Results/Properties/')
def query_url(
        n_results : int, 
        verbose : bool, 
        querystring : dict, 
        url : str,
        zzzzipcode : str,
        zzzcity : str,
        zzzsort : str
    ) -> Tuple[dict, List[dict]]:
    '''
    Honestly, this was just made the make the get_Properties function just a little less busy.
    When looking at pulling houses at the API level, we need a url, query string, the number of results, and a 
    header for the request. This takes all of that and can return properties as expected for the remsinder of the 
    program.
    '''

    v2 : bool = 'v2' in url

    response = requests.request("GET", url, headers=USREALESTATE_API_HEADERS, params=querystring).json()
    total_houses_available = response['data']['home_search']['total'] if v2 else response['data']['total']
    total_houses_in_request = response['data']['home_search']['count'] if v2 else response['data']['count'] 

    total_houses_available = int(total_houses_available)
    total_houses_in_request = int(total_houses_in_request)

    if n_results is None:
        n_results = total_houses_available

    if verbose:
        print(f'Returning {str(min(total_houses_available, n_results))} \
                out of a possible {str(total_houses_available)}.')

    geo_to_return = response['data']['geo'] if v2 else {}
    houses_to_return = response['data']['home_search']['results'] if v2 else response['data']['results']

    houses_remaining = min(total_houses_available, n_results) - len(houses_to_return)

    while houses_remaining > 0:
        querystring['offset'] = str(int(querystring['offset']) + RESULTS_PER_REQUEST_LIMIT)
        querystring['limit'] = str(min(RESULTS_PER_REQUEST_LIMIT, houses_remaining))

        response = requests.request("GET", url, headers=USREALESTATE_API_HEADERS, params=querystring).json()
        if v2:
            houses_to_return.extend(response['data']['home_search']['results'])
            houses_remaining -= len(response['data']['home_search']['results'])
        else:
            houses_to_return.extend(response['data']['results'])
            houses_remaining -= len(response['data']['results'])

    return geo_to_return, houses_to_return

@beartype
def get_UserHome(
        user_input : str
    ) -> dict:

    location_suggest = get_LocationSuggest(user_input, return_all=True)
    locations = location_suggest.get('data')

    if locations is None:
        raise Exception('Nothing returned based on query.')

    filtered_locations = [l for l in locations if l.get('_score', 0) > 20 and l.get('full_address') is not None]

    if len(filtered_locations) > 1:
        print(f'We have found {len(filtered_locations)} plausible locations for your entry:')
        
        for i, e in enumerate(filtered_locations):
            print(f'#{i+1} : {e["full_address"][0]}')
        
        house_choice : str = html.escape(
            input('Do any of these look good? If yes, enter the # or enter "no":')
        )

        if house_choice == 'no':
            raise Exception('No valid house located.')

        try:
            return filtered_locations[int(house_choice.strip()) - 1]
        except:
            raise Exception(f'Your selection was not acceptable: {house_choice}')
    
    return filtered_locations[0]

def get_HousesOfInterest(
    address              : dict, 
    n                    : int = 84, 
    listed_to_sold_ratio : float = 0.3,
    verbose              : bool = False
) -> dict:
    '''
    This function is going to take an address and retrun a dictionary of geo / results. This is going to be a gloified 
    wrapper of the API call functions, aggregating the outputs.

    address - This is a valid address, expected to be sourced from the get_UserHome.

    n - How many overall results are there going to be.
    
    listed_to_sold_ratio - This model will be a combination of listed and sold houses. T
        his will be dictated by overall reasults, this may or may not be removed based on how it works 
        when were futher down the line.

    This function is currently implemented to only look at the city_state queries. This might be ammended in future iterations
    to get some focus on the strictly local samples, but as of right now, zip_codes are not utilized.
    '''
    n_listed = int(n * listed_to_sold_ratio)
    n_sold = int(n) - n_listed
    
    # I am just going to go with city (for now)
    # TODO: Future enhancement, be able to pass a ratio of zip code and city, but not sure given I measure distance later.
    # 

    listed_homes = get_Properties(
        market_status='for_sale',
        city_state=(address.get('city'), address.get('state_code')),
        n_results=n_listed
    )

    sold_homes = get_Properties(
        market_status='sold',
        city_state=(address.get('city'), address.get('state_code')),
        n_results=n_sold
    )

    if len(listed_homes['houses']) < n_listed:
        new_n_results = n_listed-len(listed_homes['houses']) 
        if verbose:
            print(f'Shortfall in listed houses detected, appending {str(new_n_results)} of current listing to results.')

        sold_homes_v2 = get_Properties(
            market_status='sold',
            city_state=(address.get('city'), address.get('state_code')),
            n_results=n_listed - len(listed_homes['houses']),
            offset=new_n_results
        )

        sold_homes['houses'].append(sold_homes_v2['houses'])

    if len(sold_homes['houses']) < n_sold:
        new_n_results = n_sold-len(sold_homes['houses'])
        if verbose:
            print(f'Shortfall in listed houses detected, appending {str(new_n_results)} of current listing to results.')

        listed_homes_v2 = get_Properties(
            market_status='for_sale',
            city_state=(address.get('city'), address.get('state_code')),
            n_results=n_listed,
            offset=new_n_results
        )
        
        listed_homes['houses'].extend(listed_homes_v2['houses'])

    # Geo will be the same for both unless one returns a zip_code not in the other, but if that happens 
    # it will be a small population.
    
    # NOTE: Listed home always goes first as long as there is a non zero listed home query, which are both v2, so the 
    # Geo not available in v1 issue should not be an problem.
    listed_homes['houses'].extend(sold_homes['houses'].copy())

    return listed_homes

class geo_data():
    '''
    This is going to be used to organize the meta information about each query.
    I need to think where it is most appropriate to do this.
    '''
    def __init__(self, 
            stats : dict
        ):
        
        self.zip_info = self._parse_areas(stats.get('recommended_zips', {}).get('geos'))
        self.city_info = self._parse_areas(stats.get('recommended_cities', {}).get('geos'))
        self.county_info = self._parse_areas(stats.get('recommended_counties', {}).get('geos'))
        self.neighborhood_info = self._parse_areas(stats.get('recommended_neighborhoods', {}).get('geos'))
        self.market_stats = self._parse_statistics(stats.get('geo_statistics', {}).get('housing_market'))

    def __repr__(self) -> str:
        pass

    def _parse_areas(self, geos : dict) -> dict:
        return None if geos is None else {
            v.get(v.get('geo_type', 'slug_id'), '_parse_areas_FAILED') : {
                'slug_id' : v.get('slug_id'),
                'median_listing_price' : v.get('geo_statistics', {}).\
                    get('housing_market', {}).\
                    get('median_listing_price'),
                'state_code' : v.get('state_code'),
                'city_code' : v.get('city'),
                'geo_type' : v.get('geo_type')
            } for v in geos
        }
    
    def _parse_statistics(self, geo_stats : dict) -> dict:
        return None if geo_stats is None else {
            'median_days_on_market' : geo_stats.get('median_days_on_market'),
            'median_sold_price' : geo_stats.get('median_sold_price'),
            'median_price_per_sqft' : geo_stats.get('median_price_per_sqft'),
            'median_listing_price' : geo_stats.get('median_listing_price'),
            'month_to_month_metrics' : geo_stats.get('month_to_month'),
            'by_prop_type' : {
                ht.get('type') : {
                    k : v for k, v in ht.get('attributes', {}).items()
                } for ht in geo_stats.get('by_prop_type', {})
            }
        }

class house():
    '''
    This is going to be the class that houses (hehe) all the house data. Each house will have its own instance.
    When we use the API, there is a lot of data reutned nested in a number of dictionaries. This will take the 'juicy' bit.
    The idea for this class is that it will hold all the needed info for:
         1) the GUI, address, google street view, other photos. This will probably be a flask application to start, but we are 
            far from even thinking about that.
         2) the MODEL, tags, list_prices, other flags. What if we created a word cloud and have the user select key words for 
            their house until they have selected some flat number or % contribution to model from the tags TBD. There will be 
            dates there, we will use days old (or something similar) for the model training, while the actual took will use zero, 
            as the user is entering 100% correct info. This may or may not be a good idea, as it might have unintended implications
            within the model.

    Interior functions:
        Date Cleaning
        Location Cleaning
        Description Cleaning
    '''
    def __init__(self, 
            listing : dict
        ):
        
        self.reference_info = { # This is stuff not going into the model
            'id' : listing.get('property_id', ''),
            'photos' : 'to be added'
        }

        self.raw_last_update : str = listing['last_update_date']
        self.raw_list_date : str = listing['list_date']
        self.tags : list = listing.get('tags', [])
        self.price : Tuple[int, float] = listing.get('list_price', listing.get('sold_price', 0))
        self.new_construction : bool = listing.get('flags', {}).get('is_new_construction', False) or False
        self.status : Literal['sold', 'for_sale', 'NO_STATUS'] = listing.get('status', 'NO_STATUS')

        self.raw_location : dict = listing['location']
        self.raw_description : dict = listing['description']

        self._clean_dates()
        self._clean_location()
        self._clean_description()

        # This is going to be used to store stuff in the future.
        self.future_stats = {}
        self.features = {}

    def __repr__(self) -> str:
        pass
        
    def _convert_date(self, date : str) -> datetime:
        return datetime.strptime(date, '%Y-%m-%d')
    
    def _clean_dates(self) -> None:
        last_update_date_parsed = self.raw_last_update.split('T')
        list_date_parsed = '' if self.status == 'sold' else self.raw_list_date.split('T')
        self.last_update = self._convert_date(last_update_date_parsed[0]) if len(last_update_date_parsed) == 2 else None
        self.list_date = self._convert_date(list_date_parsed[0]) if len(list_date_parsed) == 2 else None
        self.last_update_delta = None if self.last_update is None else max((datetime.now() - self.last_update).days, 0)
        self.list_date_delta = None if self.list_date is None else max((datetime.now() - self.list_date).days, 0)
        
    def _clean_location(self) -> None:
        self.reference_info.update({
            'zip_code' : self.raw_location.get('address', {}).get('postal_code'),
            'state' : self.raw_location.get('address', {}).get('state'),
            'google_map_street_view' : self.raw_location.get('street_view_url'),
            'fips_code' : self.raw_location.get('county', {}).get('fips_code'),
            'county' : self.raw_location.get('county', {}).get('name'),
            'city' : self.raw_location.get('address', {}).get('city'),
        })

        lat_long = self.raw_location.get('address', {}).get('coordinate')
        self.lat_long = (0, 0) if lat_long is None else (lat_long.get('lat', 0), lat_long.get('lon', 0))     

    def _clean_description(self) -> None:
        self.baths_full = self.raw_description.get('baths_full') or 0
        self.baths_3qtr = self.raw_description.get('baths_3qtr') or 0
        self.baths_half = self.raw_description.get('baths_half') or 0
        self.baths_1qtr = self.raw_description.get('baths_1qtr') or 0
        self.year_built = self.raw_description.get('year_built') or 0
        self.lot_sqft = self.raw_description.get('lot_sqft') or 0
        self.sqft = self.raw_description.get('sqft') or 0
        self.garage = self.raw_description.get('garage') or 0
        self.stories = self.raw_description.get('stories') or 1
        self.beds = self.raw_description.get('beds') or 0
        self.type = self.raw_description.get('type') or 'NONE'

    def validate(self) -> None:
        '''
        This is just going to look at the data we will need at the next step and see if its mostly good.
        '''
        validation_dict = {
            'Days_listed' : self.list_date_delta,
            'Days_updated' : self.last_update_delta,
            'baths_full' : self.baths_full,
            'baths_3qtr' : self.baths_3qtr,
            'baths_half' : self.baths_half,
            'baths_1qtr' : self.baths_1qtr,
            'year_built' : self.year_built,
            'lot_sqft' : self.lot_sqft,
            'sqft' : self.sqft,
            'garage' : self.garage,
            'stories' : self.stories,
            'beds' : self.beds,
            'type' : self.type,
            'tags' : self.tags,
            'new_construction' : self.new_construction
        }

        for k, v in validation_dict.items():
            if v is None:
                print(f'{k} missing value for house {self.reference_info.id}')

# This will take in house and geo, and generate stats based on what is fed, and then can output a dictionary that 
# can easily be converted into a pd.Dataframe for the pipeline.
class FeatureGenerator(BaseEstimator, TransformerMixin):
    def __init__(self, 
        houses    : List[house], 
        gd        : geo_data,
        user_home : house
        ):

        '''
        This is the first step in the pipeline.
        This will import a list of houses, the geo data, and the user house

         - Look at distances between coordinates, NOT ZIPCODES
         - Need to look at listing price / median price of the zipcode.
        '''

        self.houses = houses
        self.gd = gd
        self.user_home = user_home

        self.user_home_lat_lon  = (
            self.user_home.get('centroid', {}).get('lat', 0),
            self.user_home.get('centroid', {}).get('lon', 0)
        )

        # I need to keep in mind here that this is a list of houses
        # I will need to do do these functions in  some sort of apply or list comprehension
        self.houses = list(map(self._generate_distance_between_coordinates, self.houses))
        
        self.features = list(map(self._generate_features, self.houses))
        self.targets = list(map(self._generate_targets, self.houses))

    def __repr__(self) -> str:
        pass

    def _generate_distance_between_coordinates(self, h : house) -> house:
        user_loc : Tuple[float, float] = self.user_home_lat_lon
        query_loc : Tuple[float, float] = h.lat_long

        lat1, lon1 = radians(user_loc[0]), radians(user_loc[1])
        lat2, lon2 = radians(query_loc[0]), radians(query_loc[1])

        # Haversine formula
        dlon, dlat = lon2 - lon1, lat2 - lat1
        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2

        c = 2 * asin(sqrt(a))
        r = 6371 # Radius of earth in kilometers. Use 3956 for miles
        h.future_stats['distance_from_user_home'] =  c * r
        return h

    def _generate_features(self, h) -> dict:
        h.features = {
            'Days_listed' : int(h.list_date_delta or 0),
            'Days_updated' : int(h.last_update_delta or 0),
            'baths_full' : int(h.baths_full),
            'baths_3qtr' : int(h.baths_3qtr),
            'baths_half' : int(h.baths_half),
            'baths_1qtr' : int(h.baths_1qtr),
            'year_built' : int(h.year_built),
            'lot_sqft' : int(h.lot_sqft),
            'sqft' : int(h.sqft),
            'garage' : int(h.garage),
            'stories' : int(h.stories),
            'beds' : int(h.beds),
            'type' : str(h.type),
            'tags' : h.tags or [],
            'new_construction' : bool(h.new_construction),
            'distance_to_home' : 1 / (1 + float(h.future_stats.get('distance_from_user_home', 0) or 0))
        }
        
        return h.features
    
    def _generate_targets(self, h) -> int:
        return int(h.price or 0)


In [92]:
user_home = get_UserHome(keys['SampleHouse'].strip("\'"))
hoi = get_HousesOfInterest(user_home, n=42, listed_to_sold_ratio=0.3, verbose=True)
gd = geo_data(hoi['geo'])

ValueError: Invalid placeholder in string: line 1, col 1

In [30]:
xx = house(hoi['houses'][0])
xx.lat_long

(47.551367, -122.270339)

In [33]:
user_home.get('centroid', {})

{'lon': -122.306358, 'lat': 47.572613}

In [35]:
fg = FeatureGenerator(
    houses = [house(h) for h in hoi['houses']], 
    gd=gd,
    user_home=user_home
)

In [36]:
# Basic transformer to dataframe.
class ToDataFrame(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return pd.DataFrame(X)

In [37]:
pipe = Pipeline([
    ('format_data', ToDataFrame())
])

output = pipe.fit_transform(fg.features, fg.targets)

In [38]:
to_data_frame = ToDataFrame()
X_t = to_data_frame.fit_transform(fg.features)
X_t

,Days_listed,Days_updated,baths_full,baths_3qtr,baths_half,baths_1qtr,year_built,lot_sqft,sqft,garage,stories,beds,type,tags,new_construction,distance_to_home
0,0,0,3,0,0,0,1913,5150,1830,0,1,3,single_family,"[central_air, community_outdoor_space, dining_...",False,0.217883
1,0,0,1,0,0,0,1950,5286,1680,1,1,3,single_family,"[dining_room, fireplace, forced_air, hardwood_...",False,0.093110
2,0,0,2,0,0,0,1915,4254,1550,0,1,3,single_family,"[dining_room, dishwasher, family_room, forced_...",False,0.060927
3,0,0,2,0,0,0,1970,5054,1706,2,1,5,single_family,"[community_outdoor_space, dining_room, dishwas...",False,0.325083
4,0,0,1,2,0,0,1928,3440,2020,1,1,4,single_family,"[community_outdoor_space, community_security_f...",False,0.067129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,67,1,1,1,0,1910,5000,3198,0,1,4,single_family,"[community_clubhouse, community_outdoor_space,...",False,0.079253
996,0,68,1,0,0,0,1909,6900,2280,1,1,3,single_family,"[carport, community_outdoor_space, dishwasher,...",False,0.128664
997,0,67,1,1,0,0,1944,6656,1360,1,1,3,single_family,"[community_outdoor_space, dining_room, dishwas...",False,0.074788
998,0,67,1,2,1,0,2008,1312,1580,1,1,3,single_family,"[city_view, community_outdoor_space, dishwashe...",False,0.361192


In [39]:
X_t['targets'] = fg.targets

In [40]:
X_t[X_t['targets'] < 100000]

,Days_listed,Days_updated,baths_full,baths_3qtr,baths_half,baths_1qtr,year_built,lot_sqft,sqft,garage,stories,beds,type,tags,new_construction,distance_to_home,targets
336,0,0,1,0,0,0,1919,7425,790,0,1,2,single_family,[],False,0.161811,0
342,0,0,1,0,0,0,1961,10352,1020,0,1,3,single_family,[],False,0.103236,0
349,0,0,1,0,0,0,1927,3000,1220,0,2,3,single_family,[],False,0.083717,0
351,0,0,1,0,1,0,1959,10240,1920,0,1,4,single_family,[],False,0.101978,0
360,0,0,1,0,0,0,1949,7336,720,0,1,2,single_family,[],False,0.107203,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
974,0,0,2,0,0,0,2002,6885,2270,0,2,2,single_family,[],False,0.059402,0
987,0,0,3,1,0,0,1987,3867,1240,0,1,4,single_family,[],False,0.058843,0
989,0,0,2,1,0,0,1972,4650,1180,0,1,4,single_family,[],False,0.067448,0
990,0,0,2,1,0,0,1972,6350,1250,0,1,3,single_family,[],False,0.064243,0


In [43]:
fg.houses[366].raw_description

{'year_built': 1941,
 'baths_3qtr': None,
 'sold_date': '2022-12-20',
 'sold_price': 1098500,
 'baths_full': 2,
 'name': None,
 'baths_half': None,
 'lot_sqft': 4224,
 'sqft': 2600,
 'baths': 2,
 'sub_type': None,
 'baths_1qtr': None,
 'garage': 1,
 'stories': None,
 'beds': 4,
 'type': 'single_family'}

In [ ]:
vv = fg.houses.copy()
kk = [v for v in vv if v.year_built == 1900]